![USD](usd.png)

In [23]:
#Library
import mysql.connector
from mysql.connector import errorcode

# Initial Connection

In [24]:
#Database
database = 'sd_pd_calls'

In [25]:
#Initial Connection
cnx = mysql.connector.connect(user='root',
                              password='Password1',
                              host='localhost',
                              )
cursor = cnx.cursor()

# Drop if necessary

In [4]:
#Drop DB
#mycursor = cnx.cursor()
#mycursor.execute("DROP DATABASE " + database)


# DB Setup

In [26]:
def Create_DB(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(database))
    print("Using {}".format(database))
except mysql.connector.Error as err:
    print("Database {} does not exists. Will Create".format(database))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        Create_DB(cursor)
        print("Database {} created successfully.".format(database))
        cnx.database = database
    else:
        print(err)
        exit(1)

Using sd_pd_calls


# Tables

TABLES['Days'] = (
    "CREATE TABLE `Days` ("
    "  `day_of_week` enum('1','2','3','4','5','6','7') NOT NULL,"
    "  `day` varchar(9) NOT NULL,"
    "  PRIMARY KEY (`day_of_week`)"
    ") ")

In [27]:
#Tables

#initiate dictionary
TABLES = {}

#Add tables
TABLES['Beats'] = (
    "CREATE TABLE `Beats` ("
    "  `beat` smallint NOT NULL,"
    "  `neighborhood` varchar(25) NOT NULL,"
    "  PRIMARY KEY (`beat`), UNIQUE KEY `neighborhood` (`neighborhood`)"
    ") ")

TABLES['Dispositions'] = (
    "CREATE TABLE `Dispositions` ("
    "  `dispo_code` varchar(3) NOT NULL,"
    "  `description` tinytext NOT NULL,"
    "  PRIMARY KEY (`dispo_code`)"
    ") ")

TABLES['Call_type'] = (
    "CREATE TABLE `Call_types` ("
    "  `call_type` varchar(10) NULL,"
    "  `description` tinytext NOT NULL"
    ") ")

TABLES['ServiceCalls'] = (
    "CREATE TABLE `Service_calls` ("
    "  `incident_num` varchar(13) NOT NULL,"
    "  `incident_date_time` datetime NOT NULL,"
    "  `call_type` varchar(10) NULL,"
    "  `dispo_code` varchar(3) NULL,"
    "  `beat` smallint NOT NULL,"
    "  `priority` tinyint NOT NULL,"
    "  `incident_year` year NOT NULL,"
    "  PRIMARY KEY (`incident_num`),"
    "  CONSTRAINT `service_calls_beat` FOREIGN KEY (`beat`)"
    "     REFERENCES `Beats` (`beat`),"
    "  CONSTRAINT `service_calls_dispo` FOREIGN KEY (`dispo_code`)"
    "     REFERENCES `Dispositions` (`dispo_code`)"
    ") ")

TABLES['Address'] = (
    "CREATE TABLE `Address` ("
    "  `incident_num` varchar(13) NOT NULL,"
    "  `address_number_primary` int(8)  NULL,"
    "  `address_dir_primary` varchar(10)  NULL,"
    "  `address_road_primary` varchar(25) NULL,"
    "  `address_sfx_primary` varchar(10) NULL,"
    "  `address_dir_intersecting` varchar(10) NULL,"
    "  `address_road_intersecting` varchar(10) NULL,"
    "  `address_sfx_intersecting` varchar(10) NULL,"
    "  `address` varchar(80) NOT NULL,"
    "  PRIMARY KEY (`incident_num`),"
    "  CONSTRAINT `Address_ibfk_1` FOREIGN KEY (`incident_num`)"
    "     REFERENCES `Service_calls` (`incident_num`)"
    ") ")

TABLES['GeoLocations'] = (
    "CREATE TABLE `GeoLocations` ("
    "  `incident_num` varchar(13) NOT NULL,"
    "  `lat` Decimal(8,6) NOT NULL,"
    "  `lng` Decimal(9,6) NOT NULL,"
    "  PRIMARY KEY (`incident_num`),"
    "  CONSTRAINT `GeoLocations_ibfk_1` FOREIGN KEY (`incident_num`)"
    "     REFERENCES `Service_calls` (`incident_num`)"
    ") ")

TABLES['Zipcodes'] = (
    "CREATE TABLE `Zipcodes` ("
    "  `incident_num` varchar(13) NOT NULL,"
    "  `Zipcode` MEDIUMINT(5) NOT NULL,"
    "  PRIMARY KEY (`incident_num`),"
    "  CONSTRAINT `Zipcodes_ibfk_1` FOREIGN KEY (`incident_num`)"
    "     REFERENCES `Service_calls` (`incident_num`)"
    ") ")



# Views

In [37]:
#Views

#initiate dictionary
VIEWS = {}

#Add VIEWS
VIEWS['CallsPerDayByReasonDispositionBeat'] = (
    "CREATE VIEW `CallsPerDayByReasonDispositionBeat` AS ( "
        "SELECT DATE(SC.INCIDENT_DATE_TIME) AS Incident_Date, "
        "CT.DESCRIPTION AS Reason_For_Call, "
        "D.DESCRIPTION AS Disposition, "
        "B.NEIGHBORHOOD AS Beat, COUNT(*) AS Total_Incidents "
        "FROM SERVICE_cALLS SC "
        "INNER JOIN CALL_TYPES CT "
        "    ON SC.CALL_TYPE = CT.CALL_TYPE "
        "INNER JOIN DISPOSITIONS D "
        "    ON SC.DISPO_CODE = D.DISPO_CODE "
        "INNER JOIN BEATS B ON SC.BEAT = B.BEAT "
        "GROUP BY DATE(SC.INCIDENT_DATE_TIME), CT.DESCRIPTION, D.DESCRIPTION, B.NEIGHBORHOOD "
    ") ")

VIEWS['CallsPerZipCode'] = (
    "CREATE VIEW `CallsPerZipCode` AS ( "
        "SELECT ZC.ZIPCODE, COUNT(*) AS CALLSPERZIPCODE "
        "FROM SERVICE_CALLS SC INNER JOIN ZIPCODES ZC "
        "ON SC.INCIDENT_NUM = ZC.INCIDENT_NUM "
        "GROUP BY ZC.ZIPCODE "
    ") ")

VIEWS['CallTypesPerZipCode'] = (
    "CREATE VIEW `CallTypessPerZipCode` AS ( "
        "SELECT ZC.ZIPCODE, COUNT(DISTINCT SC.CALL_TYPE) AS CALLTYPESSPERZIPCODE "
        "FROM SERVICE_CALLS SC INNER JOIN ZIPCODES ZC "
        "ON SC.INCIDENT_NUM = ZC.INCIDENT_NUM "
        "GROUP BY ZC.ZIPCODE "
    ") ")

VIEWS['CallsByBeats'] = (
    "CREATE VIEW `CallsByBeats` AS ( "
        "SELECT B.NEIGHBORHOOD AS Beat, COUNT(DISTINCT SC.INCIDENT_NUM) AS CALLSPERBEAT "
        "FROM SERVICE_CALLS SC INNER JOIN BEATS B "
        "ON SC.BEAT = B.BEAT "
        "GROUP BY B.NEIGHBORHOOD "
    ") ")

VIEWS['CallsByCallType'] = (
    "CREATE VIEW `CallsByCallType` AS ( "
        "SELECT CT.DESCRIPTION AS CALL_TYPE, COUNT(DISTINCT SC.INCIDENT_NUM) AS CALLSPERCALLTYPE "
        "FROM SERVICE_CALLS SC INNER JOIN CALL_TYPES CT "
        "ON SC.CALL_TYPE = CT.CALL_TYPE "
        "GROUP BY CT.DESCRIPTION "
    ") ")

VIEWS['CallsByDispositionCode'] = (
    "CREATE VIEW `CallsByDispositionCode` AS ( "
        "SELECT D.DESCRIPTION AS DISPOSITION, COUNT(DISTINCT SC.INCIDENT_NUM) AS CALLSPERDISPOSITION "
        "FROM SERVICE_CALLS SC INNER JOIN DISPOSITIONS D "
        "ON SC.DISPO_CODE = D.DISPO_CODE "
        "GROUP BY D.DESCRIPTION "
    ") ")

In [29]:
cursor = cnx.cursor()

for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")


Creating table Beats: already exists.
Creating table Dispositions: already exists.
Creating table Call_type: already exists.
Creating table ServiceCalls: already exists.
Creating table Address: already exists.
Creating table GeoLocations: already exists.
Creating table Zipcodes: already exists.


In [38]:
for view_name in VIEWS:
    view_description = VIEWS[view_name]
    try:
        print("Creating view {}: ".format(view_name), end='')
        cursor.execute(view_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")


Creating view CallsPerDayByReasonDispositionBeat: already exists.
Creating view CallsPerZipCode: already exists.
Creating view CallTypesPerZipCode: already exists.
Creating view CallsByBeats: already exists.
Creating view CallsByCallType: OK
Creating view CallsByDispositionCode: already exists.


In [ ]:
cursor.close()
cnx.close()